In [16]:
! ls /usr/local/

bin    cuda	cuda-11.8  games	       include	lib64	   man	 share
colab  cuda-11	etc	   _gcs_config_ops.so  lib	licensing  sbin  src


In [17]:
! nvcc --version # nvcc compiler version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [18]:
%%writefile VectorAddition.cu
#include<stdio.h>
#include<time.h>
#include<stdlib.h>
#include<cuda_runtime.h>

// Kernel Function - GPU function that is meant to be called from CPU code
__global__ void VectorAddition(float *c_d, float *a_d, float *b_d, int N){
 int idx = blockIdx.x * blockDim.x + threadIdx.x;
 if (idx<N){
  c_d[idx] = a_d[idx] + b_d[idx];
 }
}

// Main code executed by the host
int main(void){
  float *a_h, *b_h, *c_h; // pointers host
  float *a_d, *b_d, *c_d; // pointers device
  const int N = 24; // No. of elements in arrays

  size_t size=N * sizeof(float);

  a_h = (float *)malloc(size);
  b_h = (float *)malloc(size);
  c_h = (float *)malloc(size);

  srand(time(NULL));
  for(int i = 0; i < N; i++){
    a_h[i] = rand() % 100 + 1.0;
    b_h[i] = rand() % 100 + 1.0;
  }

  printf("\nArray a:\n");
  for(int i = 0; i < N; i++) printf("%f ", a_h[i]);
  printf("\nArray b:\n");
  for(int i = 0; i < N; i++) printf("%f ", b_h[i]);

  cudaMalloc((void **) &a_d, size);
  cudaMalloc((void **) &b_d, size);
  cudaMalloc((void **) &c_d, size);

  cudaMemcpy(a_d, a_h, size, cudaMemcpyHostToDevice);
  cudaMemcpy(b_d, b_h, size, cudaMemcpyHostToDevice);

  int block_size = 8;
  int n_blocks = N/block_size + (N%block_size == 0 ? 0:1);

  VectorAddition<<<n_blocks, block_size>>>(c_d, a_d, b_d, N);

  cudaMemcpy(c_h, c_d, size, cudaMemcpyDeviceToHost);

  printf("\nArray c:\n");
  for(int i = 0; i < N; i++) printf("%f ", c_h[i]);

  free(a_h);
  free(b_h);
  free(c_h);

  cudaFree(a_d);
  cudaFree(b_d);
  cudaFree(c_d);

  return(0);

}


Overwriting VectorAddition.cu


In [19]:
! nvcc VectorAddition.cu -o test

In [20]:
! ./test


Array a:
28.000000 77.000000 98.000000 93.000000 94.000000 42.000000 77.000000 46.000000 70.000000 70.000000 55.000000 54.000000 26.000000 52.000000 12.000000 30.000000 4.000000 85.000000 25.000000 30.000000 39.000000 66.000000 94.000000 81.000000 
Array b:
81.000000 95.000000 68.000000 77.000000 10.000000 96.000000 24.000000 14.000000 25.000000 93.000000 36.000000 87.000000 16.000000 58.000000 72.000000 91.000000 6.000000 1.000000 30.000000 70.000000 71.000000 16.000000 11.000000 15.000000 
Array c:
109.000000 172.000000 166.000000 170.000000 104.000000 138.000000 101.000000 60.000000 95.000000 163.000000 91.000000 141.000000 42.000000 110.000000 84.000000 121.000000 10.000000 86.000000 55.000000 100.000000 110.000000 82.000000 105.000000 96.000000 